# Import packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import numpy as np

# 1. Prepare and merge the datasets

In [2]:
df_google_first = pd.read_csv('app_sdk/google_data_first.csv', encoding='ISO-8859-1')
df_google_first = df_google_first.loc[:, ~df_google_first.columns.str.contains('^Unnamed')]

In [3]:
df_google_last = pd.read_csv('app_sdk/google_data_last.csv', encoding='ISO-8859-1')
df_google_last = df_google_last.loc[:, ~df_google_last.columns.str.contains('^Unnamed')]

In [4]:
missing_rows = df_google_first[~df_google_first['my_app_id'].isin(df_google_last['my_app_id'])]

df = pd.concat([df_google_last, missing_rows], ignore_index=True)

# 2. Analyze dataframe

In [6]:
def dataframe_summary(df):
    print(f"DataFrame contains {df.shape[0]} rows and {df.shape[1]} columns.\n")
    df_info = pd.DataFrame({
        'Data Type': df.dtypes,
        'Non-null Count': df.count(),
        'Null Count': df.isnull().sum(),
        'Null Percentage (%)': (df.isnull().sum() / len(df)) * 100
    })
    print(df_info)

# Run the function to get an enhanced summary of the DataFrame
dataframe_summary(df)

DataFrame contains 1016666 rows and 27 columns.

                    Data Type  Non-null Count  Null Count  Null Percentage (%)
my_app_id              object         1016666           0             0.000000
date_published         object         1016663           3             0.000295
privacy_policy         object          726139      290527            28.576445
rating_app            float64          927640       89026             8.756661
nb_rating              object          927639       89027             8.756760
num_downloads          object         1016651          15             0.001475
content_rating_app     object         1016663           3             0.000295
developer              object         1016658           8             0.000787
categ_app              object         1016658           8             0.000787
in_app                   bool         1016666           0             0.000000
has_ads                  bool         1016666           0             0.000000
pri

# 2. Preprocess DataFrame

In [7]:
# Use specific columns
df = df[['my_app_id', 'num_downloads', 'rating_app', 'nb_rating', 'price_gplay', 'in_app', 'content_rating_app', 'categ_app', 'date_published', 'has_ads', 'family_library', 'developer_name', 'developer_info']]
dataframe_summary(df)

DataFrame contains 1016666 rows and 13 columns.

                   Data Type  Non-null Count  Null Count  Null Percentage (%)
my_app_id             object         1016666           0             0.000000
num_downloads         object         1016651          15             0.001475
rating_app           float64          927640       89026             8.756661
nb_rating             object          927639       89027             8.756760
price_gplay           object         1012276        4390             0.431804
in_app                  bool         1016666           0             0.000000
content_rating_app    object         1016663           3             0.000295
categ_app             object         1016658           8             0.000787
date_published        object         1016663           3             0.000295
has_ads                 bool         1016666           0             0.000000
family_library          bool         1016666           0             0.000000
developer_name 

## 2.1 Classify Apps into Levels

### 2.1.1 Preprocess price_gplay to a numerical value.

Should be noted the app's prices have different currencies.

In [176]:
# Drop rows where no numeric part is found (if desired)
df = df.dropna(subset=['price_gplay'])

In [177]:
# List of currency symbols and codes to match, including €
currency_codes = ["€", "£", "$", "USD", "CZK", "EUR", "RUB", "AUD", "GBP", "JPY", "CAD", "KRW", "INR", "AED", 
                  "DKK", "SAR", "PLN", "UAH", "SEK", "IDR", "TRY", "TWD", "ILS", "HKD", "BGN", "MXN", "MYR", 
                  "BRL", "CLP", "CHF", ""]

# Create a regex pattern to match the symbols and codes
currency_pattern = r'|'.join(map(re.escape, currency_codes))

df_filtered = df.copy()

# Remove currency symbols and codes
df_filtered['price_value'] = df_filtered['price_gplay'].apply(lambda x: re.sub(currency_pattern, '', x).strip() if isinstance(x, str) else None)

df_filtered['price_value'] = df_filtered['price_value'].str.replace(',', '.')

# remove initial-scale=1. minimum-scale " hre
df_filtered['price_value'] = df_filtered['price_value'].str.replace('initial-scale=1. minimum-scale " hre', '')

# remove empty strings
df_filtered = df_filtered[df_filtered['price_value'] != '']

# convert to float
df_filtered['price_value'] = df_filtered['price_value'].astype(float)

# make new column with if app is free or not
df_filtered['is_free'] = df_filtered['price_value'].apply(lambda x: True if x == 0 else False)

### 2.1.2 Convert in_app and has_ads to boolean.

In [178]:
# convert in_app and has_ads to boolean
df_filtered['in_app'] = df_filtered['in_app'].astype(bool)
df_filtered['has_ads'] = df_filtered['has_ads'].astype(bool)

df_filtered = df_filtered.dropna()

### 2.1.3 Define the levels.

In [179]:
# Create a DataFrame to represent the levels and their criteria
data = {
    "Level": ["0", "1", "2 (Sample)", "2 (Premium)", "3", "4", "5"],
    "Description": [
        "Helemaal geen monetization",
        "Alleen ads",
        "2 verschillende versies: sample",
        "2 verschillende versies: premium",
        "Gratis met ads, en mogelijkheid voor extra features",
        "Gratis met in-app aankopen (geen ads)",
        "Betalen voor app-download (geen ads/in-app aankopen)"
    ],
    "Criteria": [
        "`is_free == 1`, `in_app == 0`, `has_ads == 0`",
        "`is_free == 1`, `in_app == 0`, `has_ads == 1`",
        "`is_free == 1`, `in_app == 0`, `has_ads ∈ [0, 1]`",
        "`is_free == 0`, `in_app == 0`, `has_ads == 0`",
        "`is_free == 1`, `in_app == 1`, `has_ads == 1`",
        "`is_free == 1`, `in_app == 1`, `has_ads == 0`",
        "`is_free == 0`, `in_app == 0`, `has_ads == 0`"
    ]
}

df_levels = pd.DataFrame(data)
df_levels

,Level,Description,Criteria
0,0,Helemaal geen monetization,"`is_free == 1`, `in_app == 0`, `has_ads == 0`"
1,1,Alleen ads,"`is_free == 1`, `in_app == 0`, `has_ads == 1`"
2,2 (Sample),2 verschillende versies: sample,"`is_free == 1`, `in_app == 0`, `has_ads ∈ [0, 1]`"
3,2 (Premium),2 verschillende versies: premium,"`is_free == 0`, `in_app == 0`, `has_ads == 0`"
4,3,"Gratis met ads, en mogelijkheid voor extra fea...","`is_free == 1`, `in_app == 1`, `has_ads == 1`"
5,4,Gratis met in-app aankopen (geen ads),"`is_free == 1`, `in_app == 1`, `has_ads == 0`"
6,5,Betalen voor app-download (geen ads/in-app aan...,"`is_free == 0`, `in_app == 0`, `has_ads == 0`"


In [180]:
# Level 0: Helemaal geen monetization
predicate_level_0 = (df_filtered['is_free'] == 1) & (df_filtered['in_app'] == 0) & (df_filtered['has_ads'] == 0)

# Level 1: Alleen ads
predicate_level_1 = (df_filtered['is_free'] == 1) & (df_filtered['in_app'] == 0) & (df_filtered['has_ads'] == 1)

# Level 2: 2 verschillende versies
predicate_level_2_sample = (df_filtered['is_free'] == 1) & (df_filtered['in_app'] == 0) & (df_filtered['has_ads'].isin([1, 0]))
predicate_level_2_premium = (df_filtered['is_free'] == 0) & (df_filtered['in_app'] == 0) & (df_filtered['has_ads'] == 0)

# Level 3: Gratis met ads, met de mogelijkheid om te betalen voor extra features of een app zonder ads
predicate_level_3 = (df_filtered['is_free'] == 1) & (df_filtered['in_app'] == 1) & (df_filtered['has_ads'] == 1)

# Level 4: Gratis, maar met in-app aankopen voor andere features
predicate_level_4 = (df_filtered['is_free'] == 1) & (df_filtered['in_app'] == 1) & (df_filtered['has_ads'] == 0)

# Level 5: Betalen voor het downloaden van de app, hetzelfde als in level 2 premium
predicate_level_5 = predicate_level_2_premium


# level 0 - Free
df_level_0 = df_filtered[predicate_level_0]

# level 1 - In-app advertising
df_level_1 = df_filtered[predicate_level_1]

# Level 2
# Possible sample apps: Free, with or without ads, without in-app purchases
df_level_2_sample = df_filtered[predicate_level_2_sample]

# Possible premium apps: Paid, without ads, with or without in-app purchases
df_level_2_premium = df_filtered[predicate_level_2_premium]


# level 3 - Free with ads and in-app purchases
df_level_3 = df_filtered[predicate_level_3]

# level 4 - Semi-premium (freemium)
df_level_4 = df_filtered[predicate_level_4]

# level 5 - Premium
df_level_5 = df_filtered[predicate_level_5]


### 2.1.4 Get level 2.

In [181]:
# Preprocess further on developer names. This says a developer must have at least 1 app in df_level_2_sample and df_level_2_premium. This makes sense since we want to find the sample and premium apps of the "same" app.

# Find common developer names
common_developers = pd.merge(
    df_level_2_sample[['developer_name']],
    df_level_2_premium[['developer_name']],
    on='developer_name',
    how='inner'
)['developer_name'].unique()

# Filter both DataFrames to keep only rows with common developers
df_level_2_sample_filtered = df_level_2_sample[
    df_level_2_sample['developer_name'].isin(common_developers)
]
df_level_2_premium_filtered = df_level_2_premium[
    df_level_2_premium['developer_name'].isin(common_developers)
]

#### 2.1.4.1 Word similarity algorithm.

In [182]:
# The app names are in the 'my_app_id' column. We will use these to find similar words between the two lists.
list1 = df_level_2_sample_filtered['my_app_id'].tolist()
list2 = df_level_2_premium_filtered['my_app_id'].tolist()

# Function to find similar words with at least 75% similarity
def find_similar_words(list1, list2, threshold=75):
    # Combine both lists for TF-IDF feature extraction
    combined_list = list1 + list2

    # Use TF-IDF Vectorizer to transform the text into feature vectors
    vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 4))  # You can change ngram_range for more efficiency
    tfidf_matrix = vectorizer.fit_transform(combined_list)

    # Separate the TF-IDF vectors for both lists
    tfidf_list1 = tfidf_matrix[:len(list1)]
    tfidf_list2 = tfidf_matrix[len(list1):]

    # Use NearestNeighbors for finding approximate matches
    # Set the number of neighbors to a minimum of 10 or the size of list2 to limit comparisons
    n_neighbors = min(10, len(list2))
    nn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm='brute', n_jobs=-1)
    nn.fit(tfidf_list2)

    similar_pairs = []
    distances, indices = nn.kneighbors(tfidf_list1)

    # Iterate through the results to find pairs with similarity above the threshold
    for i, (dists, idxs) in enumerate(zip(distances, indices)):
        for dist, idx in zip(dists, idxs):
            similarity = (1 - dist) * 100
            if similarity >= threshold:
                similar_pairs.append((list1[i], list2[idx], similarity))

    # Create a DataFrame from the similar pairs
    df = pd.DataFrame(similar_pairs, columns=['Sample app name', 'Premium app name', 'Similarity (%)'])
    return df

# Run the function on the slices
df_similarities = find_similar_words(list1, list2)

In [183]:
df_similarities.head()

,Sample app name,Premium app name,Similarity (%)
0,Andrew.Compass.com,andrew.compass.color2,85.780677
1,Andrew.Compass.com,andrew.compass.color3,85.317104
2,Andrew.Compass.com,andrew.compass.color1,84.718042
3,Andrew.Compass.com,andrew.compass.com.yellow,82.218412
4,AutoLoanCalculatorPRO.DPsoftware.org,AutoLoanCalculatorPRO_Market.DPsoftware.org,80.783936


#### 2.1.4.2 Extra filter (prefix).

In [184]:
# If the first word of the app names is the same, we can consider them as similar

# Fill NaN values with empty strings
df_similarities['First_Word_List1'] = df_similarities['Sample app name'].fillna('').str.split('.').str[0]
df_similarities['First_Word_List2'] = df_similarities['Premium app name'].fillna('').str.split('.').str[0]

# Perform case-insensitive comparison
df_similarities['Comparison_Result'] = df_similarities['First_Word_List1'] == df_similarities['First_Word_List2']

# Filter rows where the first words match
df_similarities = df_similarities[df_similarities['Comparison_Result'] == True]

# Drop the helper column if no longer needed
df_similarities = df_similarities.drop(columns=['Comparison_Result', 'First_Word_List1', 'First_Word_List2'])


#### 2.1.4.3 Filter extra on the difference.

In [185]:
def get_simple_suffix_difference(str1, str2):
    # Find the common prefix
    prefix_length = 0
    while prefix_length < len(str1) and prefix_length < len(str2) and str1[prefix_length] == str2[prefix_length]:
        prefix_length += 1

    # Extract the differing suffixes
    suffix1 = str1[prefix_length:]
    suffix2 = str2[prefix_length:]

    # Return the difference in suffixes
    return f"{suffix1.lower()}{suffix2.lower()}"

df_similarities['Difference Score'] = df_similarities.apply(
    lambda row: get_simple_suffix_difference(row['Sample app name'], row['Premium app name']),
    axis=1
)


In [186]:
df_similarities.tail()

,Sample app name,Premium app name,Similarity (%),Difference Score
22338,xyz.cryptomines.spacepools,xyz.cryptomines.SpacepoolsMonitor,87.178177,spacepoolsspacepoolsmonitor
22339,xyz.cryptomines.zpool,xyz.cryptomines.ZpoolMonitor,87.264442,zpoolzpoolmonitor
22340,xyz.digigem.gem.verge,xyz.digigem.gem.diamond,79.579567,vergediamond
22341,yukod.science.electronconfiglite,yukod.science.electronconfigengine,84.524221,liteengine
22342,yukod.science.electronconfiglite,yukod.science.electronconfiguration,83.404048,liteuration


In [ ]:
# Get the tail of the second word (alphabetic characters only) that starts right after the first word
def get_tail_cleaned(first_app_name, second_app_name):
    """
    Returns the cleaned tail of the second app name (alphabetic characters only).
    that starts right after the first app name.
    """
    if first_app_name.startswith(second_app_name):
        tail = first_app_name[len(second_app_name):]
        # Remove non-alphabetic characters using regex
        return re.sub(r'[^a-zA-Z]', '', tail)
    return None

# Function to process a list of cleaned tails
def process_tails(df, col1, col2):
    """
    Extracts, cleans, and processes the tails between two columns in a DataFrame.
    Removes None and empty strings.
    """
    tails = df.apply(lambda row: get_tail_cleaned(row[col1], row[col2]), axis=1).dropna()
    return [x for x in tails if x]

# Process both sample and premium lists
sample_list = process_tails(df_similarities, 'Sample app name', 'Premium app name')
premium_list = process_tails(df_similarities, 'Premium app name', 'Sample app name')

# Count values that occur more than once
grouped_sample = pd.Series(sample_list).value_counts()
grouped_sample = grouped_sample[grouped_sample > 1]
grouped_premium = pd.Series(premium_list).value_counts()
grouped_premium = grouped_premium[grouped_premium > 1]

In [188]:
# List of indicative strings for free/sample and paid/premium versions. Inspired by grouped_sample and grouped_premium.
indicative_terms = [
    # Free/sample version terms
    "free",
    "lite",
    "demo",
    "trial",
    "beta",
    "ad",
    "light",
    "test",
    "eval",
    "preview",
    "gratis",
    "libre",
    "sample",
    "mini",
    "basic",
    "limited",
    "trail",
    "try",

    # Paid/premium version terms
    "pro",
    "full",
    "paid",
    "plus",
    "premium",
    "noads",
    "donate",
    "adfree",
    "key",
    "unlock",
    "license",
    "vip",
    "deluxe",
    "gold",
    "elite",
    "prime",
    "unlimited",
    "buy",
    "removeads"
]

In [189]:
# Filter the DataFrame to keep rows where 'Difference Score' contains any of the indicative terms
df_similarities = df_similarities[df_similarities['Difference Score'].str.contains('|'.join(indicative_terms), case=False, na=False)]

#### 2.1.4.4 Filter on level 2 sample apps and premium apps.

In [190]:
# Remove duplicate based on 'Sample app name' and 'Premium app name' columns
df_similarities = df_similarities.drop_duplicates(subset=['Sample app name'])
df_similarities = df_similarities.drop_duplicates(subset=['Premium app name'])

In [191]:
sample_app_names = df_similarities['Sample app name'].to_list()
premium_app_names = df_similarities['Premium app name'].to_list()

In [192]:
df_level_2_sample = df_level_2_sample_filtered[df_level_2_sample_filtered['my_app_id'].isin(sample_app_names)]
df_level_2_premium = df_level_2_premium_filtered[df_level_2_premium_filtered['my_app_id'].isin(premium_app_names)]

In [193]:
len(df_level_2_sample), len(df_level_2_premium)

(11432, 11432)

### 2.1.5 Filter apps from level 2 on level 0, 1 and 5.

In [194]:
# Filter on sample apps from level 2
df_level_0 = df_level_0[~df_level_0['my_app_id'].isin(df_level_2_sample['my_app_id'])]
df_level_1 = df_level_1[~df_level_1['my_app_id'].isin(df_level_2_sample['my_app_id'])]

# Filter on premium apps from level 2
df_level_5 = df_level_5[~df_level_5['my_app_id'].isin(df_level_2_premium['my_app_id'])]

### 2.1.6 Check that all levels have the right apps.

In [195]:
# Check for missing combinations by creating all unique combinations in the data
unique_combinations = df_filtered.groupby(['is_free', 'in_app', 'has_ads']).size()

# List of all covered combinations in levels
covered_combinations = pd.concat([
    df_level_0.groupby(['is_free', 'in_app', 'has_ads']).size(),
    df_level_1.groupby(['is_free', 'in_app', 'has_ads']).size(),
    df_level_2_sample.groupby(['is_free', 'in_app', 'has_ads']).size(),
    df_level_2_premium.groupby(['is_free', 'in_app', 'has_ads']).size(),
    df_level_3.groupby(['is_free', 'in_app', 'has_ads']).size(),
    df_level_4.groupby(['is_free', 'in_app', 'has_ads']).size(),
    df_level_5.groupby(['is_free', 'in_app', 'has_ads']).size()
])

# Identify combinations in the original data that are not covered by the levels
missing_combinations = unique_combinations[~unique_combinations.index.isin(covered_combinations.index)]

# Convert missing combinations to a DataFrame for easier readability
missing_combinations_df = missing_combinations.reset_index(name='count')
missing_combinations_df


,is_free,in_app,has_ads,count
0,False,False,True,3765
1,False,True,False,2481
2,False,True,True,532


In [196]:
data = {
    "is_free": [True, True, True, False, True, True, False],
    "in_app": [False, False, False, False, True, True, False],
    "has_ads": [False, True, [False, True], False, True, False, False],
    "level": ["0", "1", "2 (Sample)", "2 (Premium)", "3", "4", "5"]
}

df_levels = pd.DataFrame(data)
df_levels

,is_free,in_app,has_ads,level
0,True,False,False,0
1,True,False,True,1
2,True,False,"[False, True]",2 (Sample)
3,False,False,False,2 (Premium)
4,True,True,True,3
5,True,True,False,4
6,False,False,False,5


In [ ]:
# print the length of each level
print(f"Level 0: {len(df_level_0)}")
print(f"Level 1: {len(df_level_1)}")
print(f"Level 2 sample: {len(df_level_2_sample)}")
print(f"Level 2 premium: {len(df_level_2_premium)}")
print(f"Level 3: {len(df_level_3)}")
print(f"Level 4: {len(df_level_4)}")
print(f"Level 5: {len(df_level_5)}")

len(df_filtered)

Level 0: 313633
Level 1: 432264
Level 2 sample: 11432
Level 2 premium: 11432
Level 3: 73753
Level 4: 26286
Level 5: 48136


923714

In [202]:
# Assuming the length of each level is stored in variables, calculate the sum of all levels
total_length_sum = (
    len(df_level_0)
    + len(df_level_1)
    + len(df_level_2_sample)
    + len(df_level_2_premium)
    + len(df_level_3)
    + len(df_level_4)
    + len(df_level_5)
)

# Print the total sum
print(f"Total sum of all levels: {total_length_sum}")

# print the sum of missed combinations
print(missing_combinations_df['count'].sum())

# Is the total sum of all levels equal to the length of the original DataFrame?
print(total_length_sum + missing_combinations_df['count'].sum() == len(df_filtered))

Total sum of all levels: 916936
6778
True


## 2.2 Number of Downloads

In [ ]:
# Function to clean and convert to numbers
def convert_to_numeric(value):
    if pd.isna(value):
        return np.nan
    else:
        # Remove ',' and '+' and '>' and strip any whitespace
        cleaned_value = value.replace(',', '').replace('+', '').replace('>', '').strip()
        return int(cleaned_value)

# Apply the function to the DataFrame column
df_filtered['num_downloads'] = df_filtered['num_downloads'].apply(convert_to_numeric)

## 2.3 Ratings

In [ ]:
# convert 76,545 to 76545
df_filtered['nb_rating'] = df_filtered['nb_rating'].str.replace(',', '')

df_filtered['nb_rating'] = pd.to_numeric(df_filtered['nb_rating'])
df_filtered['rating_app'] = pd.to_numeric(df_filtered['rating_app'])

# Calculate the global average rating across all apps
global_mean_rating = df_filtered['rating_app'].mean()

# Define a prior weight (m)
# This is the number of ratings at which the average rating is considered reliable.
# Adjust based on your data; higher values give more weight to the global mean.
m = 50

# Calculate Bayesian average for each app
df_filtered['bayesian_average'] = (global_mean_rating * m + df_filtered['rating_app'] * df_filtered['nb_rating']) / (m + df_filtered['nb_rating'])


## 2.4 Content Rating

In [ ]:
# Applying the regex matching approach to the 'content_rating_app' column in df_content
df_filtered["content_rating_app"] = [
    match.group() if (match := re.match(r'PEGI (3|7|12|16|18)', item)) else item 
    for item in df_filtered["content_rating_app"]
]

# if content_rating_app contains 'not yet been rated', set to 'not yet been rated'
# Ensure the column is of type 'object' (which can store both strings and integers)
df_filtered['content_rating_app'] = df_filtered['content_rating_app'].astype('object')

# Now you can safely assign the string value
df_filtered.loc[df_filtered['content_rating_app'].str.contains('not yet been rated'), 'content_rating_app'] = 'not yet been rated'

## 2.5 Dates

In [ ]:
df_filtered['date_published'] = pd.to_datetime(df_filtered['date_published'], errors='coerce')

# 3. Graphs

## 3.1 Main observations

### 3.1.1 Date Published

In [ ]:
# histogram when apps are published
df_filtered['date_published'].hist(bins=20, figsize=(10, 6))
plt.xlabel('Date Published')
plt.ylabel('Frequency')
plt.title('Distribution of Date Published')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.show()

### 3.1.2 Rating

#### 3.1.2.1 Distribution of Ratings

In [ ]:
# Plot histogram
df_filtered['rating_app'].hist(bins=20, figsize=(10, 6))
plt.xlabel('App Rating')
plt.ylabel('Frequency')
plt.title('Distribution of App Rating')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.show()

#### 3.1.2.2 Distribution of Bayesian Ratings

In [ ]:
# Plot histogram
df_filtered['bayesian_average'].hist(bins=20, figsize=(10, 6))
plt.xlabel('Bayesian average App Rating')
plt.ylabel('Frequency')
plt.title('Distribution of Bayesian average App Rating')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.show()

### 3.1.3 Number of downloads

In [ ]:
# Define the bins for categorization
bins = [0, 100, 1000, 10000, 100000, 1000000, 10000000, 100000000, 1000000000, np.inf]
labels = ['0-100', '101-1k', '1k-10k', '10k-100k', '100k-1M', '1M-10M', '10M-100M', '100M-1B', '1B+']

# Categorize num_downloads into these bins
df_filtered['downloads_category'] = pd.cut(df_filtered['num_downloads'], bins=bins, labels=labels)

# Plot the distribution of num_downloads categories
plt.figure(figsize=(12, 6))
df_filtered['downloads_category'].value_counts(sort=False).plot(kind='bar', edgecolor='black')
plt.xlabel('Number of Downloads (Categories)')
plt.ylabel('Frequency')
plt.title('Distribution of Number of Downloads by Categories')
plt.xticks(rotation=45)
plt.show()

### 3.1.4 WARNING: Prices (this is not adjusted to currencies)

In [ ]:
plt.figure(figsize=(12, 8))
plt.hist(df_filtered['price_value'], bins=50, edgecolor="black", color="skyblue", range=(-1, max(df_filtered['price_value']) + 1))
plt.yscale("log")  # Log scale to better visualize frequencies

# Adding labels and title
plt.title("Distribution of Prices on Google Play")
plt.xlabel("Price")
plt.ylabel("Log Frequency")

plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

### 3.1.5 Content Ratings

In [ ]:
plt.figure(figsize=(8, 8))
content_rating_counts = df_filtered["content_rating_app"].value_counts(normalize=True) * 100  # percentage
content_rating_counts.plot(kind='pie', autopct='%1.1f%%', startangle=140)
plt.title("Proportion of Different Content Ratings")
plt.ylabel("")  # Hide y-axis label for cleaner look
plt.show()

### 3.1.6 Category of apps

In [ ]:
category_counts = df_filtered['categ_app'].value_counts()

plt.figure(figsize=(8, 8))

category_counts.plot(kind='pie', autopct='%1.1f%%', startangle=140, wedgeprops={'edgecolor': 'black'})

plt.title("Distribution of App Categories")

plt.ylabel("")  # Hide y-axis label for cleaner look

plt.show()

### 3.1.7 Apps with in-app purchases and ads

In [ ]:
in_app_percentage = df_filtered["in_app"].value_counts(normalize=True) * 100
has_ads_percentage = df_filtered["has_ads"].value_counts(normalize=True) * 100

# Creating a DataFrame for plotting
percentage_data = pd.DataFrame({
    "Feature": ["In-app Purchases", "No In-app Purchases", "Has Ads", "No Ads"],
    "Percentage": [in_app_percentage[True], in_app_percentage[False], has_ads_percentage[True], has_ads_percentage[False]]
})



# Plotting the bar plot for percentages of apps with in-app purchases and ads
plt.figure(figsize=(10, 6))
plt.bar(percentage_data["Feature"], percentage_data["Percentage"], color=['skyblue', 'lightgreen', 'coral', 'gold'], edgecolor='black')
plt.title("Percentage of Apps with In-app Purchases and Ads")
plt.xlabel("Feature")
plt.ylabel("Percentage (%)")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

### 3.1.8 Family Library Support

In [ ]:
family_library_counts = df_filtered["family_library"].value_counts()

# Plotting the count of apps supporting or not supporting the family library feature
plt.figure(figsize=(8, 6))
family_library_counts.plot(kind='bar', color=['skyblue', 'coral'], edgecolor='black')
plt.title("Count of Apps Supporting Family Library Feature")
plt.xlabel("Family Library Support")
plt.ylabel("Number of Apps")
plt.xticks([0, 1], labels=["Does Not Support", "Supports"], rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

### 3.1.9 Average app rating per category

In [ ]:
# Splitting categories into two parts for better readability
unique_categories = df_filtered["categ_app"].unique()
midpoint = len(unique_categories) // 2
categories_part1 = unique_categories[:midpoint]
categories_part2 = unique_categories[midpoint:]

# Filtering data for each part
df_category_part1 = df_filtered[df_filtered["categ_app"].isin(categories_part1)]
df_category_part2 = df_filtered[df_filtered["categ_app"].isin(categories_part2)]

# Plotting box plot for average app ratings by category (Part 1)
plt.figure(figsize=(12, 6))
df_category_part1.boxplot(column="rating_app", by="categ_app", grid=False, vert=False)
plt.title("Average App Ratings by Category (Part 1)")
plt.suptitle("")  # Remove default title to keep it clean
plt.xlabel("Average Rating")
plt.ylabel("App Category")
plt.show()

# Plotting box plot for average app ratings by category (Part 2)
plt.figure(figsize=(12, 6))
df_category_part2.boxplot(column="rating_app", by="categ_app", grid=False, vert=False)
plt.title("Average App Ratings by Category (Part 2)")
plt.suptitle("")  # Remove default title to keep it clean
plt.xlabel("Average Rating")
plt.ylabel("App Category")
plt.show()


## 3.2 Topic 2: Freemium vs Paid-For Apps observations

### 3.2.1 Subway Surfers

In [ ]:
df_filtered[df_filtered['my_app_id'] == 'com.kiloo.subwaysurf']


### 3.2.2 Downloads per price category

#### 3.2.2.1 Total downloads

In [ ]:
# Calculate the total number of downloads for each price category
total_downloads = df_filtered.groupby('price_category')['num_downloads'].sum().reset_index()

# Set up a color palette for price categories
palette = {"free": "blue", "paid": "green", "freemium": "orange"}

# Plot the total number of downloads by price category with the updated parameters
plt.figure(figsize=(10, 6))
sns.barplot(data=total_downloads, x='price_category', y='num_downloads', hue='price_category', palette=palette, dodge=False)
plt.legend([], [], frameon=False)  # Remove the legend

# Add labels and a title
plt.title("Total Number of Downloads by Price Category")
plt.xlabel("Price Category")
plt.ylabel("Total Number of Downloads")
plt.show()


#### 3.2.2.1 Average downloads

In [ ]:
# Calculate the mean number of downloads for each price category
mean_freemium = df_filtered[df_filtered['price_category'] == 'freemium']['num_downloads'].mean()
mean_free = df_filtered[df_filtered['price_category'] == 'free']['num_downloads'].mean()
mean_paid = df_filtered[df_filtered['price_category'] == 'paid']['num_downloads'].mean()


In [ ]:
# Create a dictionary for the means
means = {
    'Free': mean_free,
    'Freemium': mean_freemium,
    'Paid': mean_paid,
}

# Convert to a pandas DataFrame for easier plotting
mean_df = pd.DataFrame(list(means.items()), columns=['Price Category', 'Average Downloads'])


In [ ]:
# Set up a color palette for price categories
palette = {"Free": "blue", "Paid": "green", "Freemium": "orange"}

# Set up the plot
plt.figure(figsize=(8, 6))

# Create the bar plot with the updated parameters
sns.barplot(x='Price Category', y='Average Downloads', data=mean_df, hue='Price Category', palette=palette, dodge=False)
plt.legend([], [], frameon=False)  # Removes the legend

# Add labels and a title
plt.title('Average Number of Downloads by Price Category', fontsize=16)
plt.xlabel('Price Category', fontsize=12)
plt.ylabel('Average Number of Downloads', fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()

### 3.2.3 Rating by Price Category

In [ ]:
# Set up a color palette for price categories
palette = {"free": "blue", "paid": "green", "freemium": "orange"}

#### 3.2.3.1 Average Rating

In [ ]:
# Average Rating by Price Category
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_filtered, x='price_category', y='rating_app', hue='price_category', palette=palette, legend=False)
plt.title("Average Rating by Price Category")
plt.xlabel("Price Category")
plt.ylabel("App Rating")
plt.show()


#### 3.2.3.2 Average Bayesian Rating

In [ ]:
# Bayesian Average by Price Category
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_filtered, x='price_category', y='bayesian_average', hue='price_category', palette=palette, legend=False)
plt.title("Bayesian Average by Price Category")
plt.xlabel("Price Category")
plt.ylabel("Bayesian Average")
plt.show()


#### 3.2.3.3 Average Number of Ratings

In [ ]:
# Number of Ratings by Price Category
plt.figure(figsize=(10, 6))
sns.barplot(data=df_filtered, x='price_category', y='nb_rating', hue='price_category', palette=palette, errorbar=None, legend=False)
plt.title("Number of Ratings by Price Category")
plt.xlabel("Price Category")
plt.ylabel("Number of Ratings")
plt.show()

#### 3.2.3.4 Total Number of Ratings

In [ ]:
# Calculate the total number of ratings for each price category
total_ratings = df_filtered.groupby('price_category')['nb_rating'].sum().reset_index()

# Plot the total number of ratings by price category
plt.figure(figsize=(10, 6))
sns.barplot(data=total_ratings, x='price_category', y='nb_rating', hue='price_category', palette=palette, dodge=False)
plt.legend([], [], frameon=False)  # Remove the legend if not needed

# Add labels and a title
plt.title("Total Number of Ratings by Price Category")
plt.xlabel("Price Category")
plt.ylabel("Total Number of Ratings")
plt.show()

### 3.2.4 TODO: Outliers

### 3.2.5 TODO: Distribution of free, freemium, paid. Average downloads per category



# 4. Merge SDK dataset

In [ ]:
# read app_sdk/app_sdk.csv
app_sdk = pd.read_csv('app_sdk/app_sdk.csv')
app_sdk = app_sdk.loc[:, ~app_sdk.columns.str.contains('^Unnamed')]

In [ ]:
# join df and app_sdk on inner join with my_app_id and id
df_app_and_sdk = pd.merge(df_filtered, app_sdk, left_on='my_app_id', right_on='id', how='inner')

# 5. Graphs SDK

## 5.1 SDK_name counts

In [ ]:
# Plotting the counts of each SDK name
plt.figure(figsize=(10, 6))
df_app_and_sdk['sdk_type'].value_counts().plot(kind='bar')
plt.title("Counts of Each SDK Type")
plt.xlabel("SDK Type")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5.2 Top 10 used SDKs

In [ ]:
# top 10 most used SDKs by free apps
df_free = df_app_and_sdk[df_app_and_sdk['price_category'] == 'free']
df_free = df_free.dropna(subset=['sdk_name'])
df_free = df_free['sdk_name'].value_counts().head(10)
df_free

In [ ]:
# top 10 most used SDKs by freemium apps
df_freemium = df_app_and_sdk[df_app_and_sdk['price_category'] == 'freemium']
df_freemium = df_freemium.dropna(subset=['sdk_name'])
df_freemium = df_freemium['sdk_name'].value_counts().head(10)
df_freemium

In [ ]:
# top 10 most used SDKs by paid apps
df_paid = df_app_and_sdk[df_app_and_sdk['price_category'] == 'paid']
df_paid = df_paid.dropna(subset=['sdk_name'])
df_paid = df_paid['sdk_name'].value_counts().head(10)
df_paid

## 5.3 Top 10 used SDKs dropping duplicates

In [ ]:
df_new = df_app_and_sdk.drop_duplicates(subset=['my_app_id', 'sdk_name'])

In [ ]:
df_free = df_new[df_new['price_category'] == 'free']
df_freemium = df_new[df_new['price_category'] == 'freemium']
df_paid = df_new[df_new['price_category'] == 'paid']

In [ ]:
df_free['sdk_name'].value_counts().head(10)

In [ ]:
df_freemium['sdk_name'].value_counts().head(10)

In [ ]:
df_paid['sdk_name'].value_counts().head(10)